# Weather forecasting

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from google.colab import files
import tensorflow as tf
from sklearn.model_selection import train_test_split

### Data loading

In [2]:
df = pd.read_csv('Weather Data.csv',sep=",")
df.head()

,DateTime,Condition,Rain,MaxTemp,MinTemp,SunRise,SunSet,MoonRise,MoonSet,AvgWind,AvgHumidity,AvgPressure
0,02.09.2019,Partly cloudy,0.0,27,22,06:32:00,19:37:00,09:52:00,21:45:00,23,66,1012
1,01.09.2019,Partly cloudy,0.0,27,22,06:31:00,19:38:00,08:37:00,21:13:00,21,66,1011
2,31.08.2019,Light rain,0.5,26,22,06:30:00,19:40:00,07:21:00,20:40:00,22,63,1015
3,30.08.2019,Partly cloudy,0.0,27,22,06:29:00,19:42:00,06:04:00,20:05:00,20,64,1016
4,29.08.2019,Partly cloudy,0.0,27,23,06:27:00,19:43:00,04:47:00,19:26:00,24,61,1015


In [3]:
df = df[["Condition", "MaxTemp", "MinTemp", "AvgWind", "AvgHumidity","AvgPressure"]]
df.head()

,Condition,MaxTemp,MinTemp,AvgWind,AvgHumidity,AvgPressure
0,Partly cloudy,27,22,23,66,1012
1,Partly cloudy,27,22,21,66,1011
2,Light rain,26,22,22,63,1015
3,Partly cloudy,27,22,20,64,1016
4,Partly cloudy,27,23,24,61,1015


In [4]:
df.shape

(3896, 6)

### Data processing

In [5]:
df = df.dropna()

In [7]:
X = np.array(df.drop(columns=['Condition'], axis=1))
X

array([[  27,   22,   23,   66, 1012],
       [  27,   22,   21,   66, 1011],
       [  26,   22,   22,   63, 1015],
       ...,
       [   5,    3,   16,   94, 1021],
       [   4,    1,   12,   89, 1021],
       [   5,    2,   10,   90, 1027]])

In [8]:
y = pd.get_dummies(df['Condition'])
y

,Blizzard,Cloudy,Fog,Heavy rain,Light rain,Light sleet,Mist,Moderate or heavy sleet,Moderate or heavy snow showers,Moderate rain,Moderate snow,Overcast,Partly cloudy,Sunny,Torrential rain shower
0,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
2,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3891,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
3892,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
3893,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
3894,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30,shuffle=True)

## Model

### Building the model

In [10]:
NB_classes = 15
NB_features = 5
activation_func = tf.keras.activations.relu

model = tf.keras.Sequential([
    tf.keras.Input(shape=(NB_features,)),
    tf.keras.layers.Dense(64, activation=activation_func),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(64, activation=activation_func),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(NB_classes, activation=tf.keras.activations.softmax)
])


early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=3)

model.compile(optimizer="adam", loss=tf.keras.losses.categorical_crossentropy, metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                384       
                                                                 
 batch_normalization (Batch  (None, 64)                256       
 Normalization)                                                  
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 batch_normalization_1 (Bat  (None, 64)                256       
 chNormalization)                                                
                                                                 
 dropout_1 (Dropout)         (None, 64)                0

### Training the model

In [17]:
model.fit(x=X_train, y=y_train, epochs=200, validation_data=(X_test,y_test), verbose=1, shuffle=True)

Epoch 1/200
86/86 [==============================] - 0s 5ms/step - loss: 1.0240 - accuracy: 0.6351 - val_loss: 1.1454 - val_accuracy: 0.6245
Epoch 2/200
86/86 [==============================] - 0s 3ms/step - loss: 1.0342 - accuracy: 0.6318 - val_loss: 1.3578 - val_accuracy: 0.6287
Epoch 3/200
86/86 [==============================] - 0s 3ms/step - loss: 1.0260 - accuracy: 0.6447 - val_loss: 1.5193 - val_accuracy: 0.5808
Epoch 4/200
86/86 [==============================] - 0s 3ms/step - loss: 1.0125 - accuracy: 0.6447 - val_loss: 1.8095 - val_accuracy: 0.5825
Epoch 5/200
86/86 [==============================] - 0s 3ms/step - loss: 1.0199 - accuracy: 0.6410 - val_loss: 1.3689 - val_accuracy: 0.5851
Epoch 6/200
86/86 [==============================] - 0s 3ms/step - loss: 1.0224 - accuracy: 0.6392 - val_loss: 1.0510 - val_accuracy: 0.6210
Epoch 7/200
86/86 [==============================] - 0s 3ms/step - loss: 1.0108 - accuracy: 0.6469 - val_loss: 1.3655 - val_accuracy: 0.6022
Epoch 8/200
8

In [18]:
performance=model.evaluate(X_test,y_test, batch_size=32, verbose=1, steps=None, )[1] * 100
print('Final accuracy : ', round(performance), '%')

37/37 [==============================] - 0s 2ms/step - loss: 1.2396 - accuracy: 0.6133
Final accuracy :  61 %


In [19]:
pred=model.predict(np.array([[27,22,23,66,1012]]))
np.argmax(pred)

1/1 [==============================] - 0s 21ms/step


13

In [20]:
pred=model.predict(np.array([[9,6,24,63,1018]]))
np.argmax(pred)

1/1 [==============================] - 0s 22ms/step


13

In [21]:
pred=model.predict(np.array([[2,0,26,71,1033]]))
np.argmax(pred)

1/1 [==============================] - 0s 21ms/step


11

### Exporting the Model

In [22]:
model.save('model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
